In [3]:
# Implementation of doc2vec on the IMDB 5000 dataset

In [10]:
from __future__ import absolute_import, division, print_function
import multiprocessing, os, pprint, re
import nltk
import gensim.models.word2vec as word2vec
import gensim.models.doc2vec as doc2vec
import sklearn.manifold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re, string

In [5]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('movie_metadata.csv')

In [6]:
df = df.drop(df.movie_imdb_link.name, 1)
def trim_and_remove_spaces(raw):
    if pd.isnull(raw) == False:
        return raw.strip().replace(' ', '_')

df.director_name = df.director_name.apply(trim_and_remove_spaces)
df.actor_2_name = df.actor_2_name.apply(trim_and_remove_spaces)

df.actor_1_name = df.actor_1_name.apply(trim_and_remove_spaces)
df.movie_title = df.movie_title.apply(trim_and_remove_spaces)
df.actor_3_name = df.actor_3_name.apply(trim_and_remove_spaces)
df.language = df.language.apply(trim_and_remove_spaces)
df.country = df.country.apply(trim_and_remove_spaces)

In [7]:
def descretize3(df, col):
    
    if(df[col].dtype == np.float64 or df[col].dtype == np.int64):
        df[col] = pd.qcut(df[col], 3, labels=['low', 'medium', 'high'], retbins=False)
        df[col] = df[col].apply(lambda x : (str(x)).replace(" ", ""))
        
def descretize2(df, col):
    
    if(df[col].dtype == np.float64 or df[col].dtype == np.int64):
        df[col] = pd.qcut(df[col], 2, labels=['low', 'high'], retbins=False)
        df[col] = df[col].apply(lambda x : (str(x)).replace(" ", ""))
        
def descretize10(df, col):
    
    if(df[col].dtype == np.float64 or df[col].dtype == np.int64):
        df[col] = pd.qcut(df[col], 3, labels=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], retbins=False)
        df[col] = df[col].apply(lambda x : (str(x)).replace(" ", ""))

descretize3(df, df.director_facebook_likes.name)
descretize3(df, df.num_critic_for_reviews.name)
descretize3(df, df.duration.name)
descretize3(df, df.actor_3_facebook_likes.name)
descretize3(df, df.actor_1_facebook_likes.name)

descretize3(df, df.gross.name)
descretize3(df, df.num_voted_users.name)
descretize3(df, df.cast_total_facebook_likes.name)
descretize3(df, df.num_user_for_reviews.name)
descretize3(df, df.budget.name)

descretize3(df, df.actor_2_facebook_likes.name)
descretize3(df, df.imdb_score.name)
descretize2(df, df.facenumber_in_poster.name)
descretize3(df, df.title_year.name)
descretize3(df, df.aspect_ratio.name)
descretize2(df, df.movie_facebook_likes.name)

In [8]:
from sklearn.utils import shuffle
df = shuffle(df)
df

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
3609,Color,Richard_Linklater,high,medium,low,low,Brandon_Smith,high,low,Comedy|Crime|Drama,Matthew_McConaughey,Bernie,medium,high,Rick_Dial,low,funeral|stockbroker|texas|vacation|widow,medium,English,USA,PG-13,low,high,low,medium,low,high
1597,Color,Steve_Oedekerk,low,low,high,medium,Bob_Gunton,low,high,Adventure|Comedy,Bruce_Spence,Ace_Ventura:_When_Nature_Calls,high,medium,Sophie_Okonedo,low,africa|bat|hawaiian shirt|pregnancy|pregnant,medium,English,USA,PG-13,medium,low,medium,medium,medium,low
521,Color,Pierre_Coffin,high,medium,high,high,Miranda_Cosgrove,high,high,Animation|Comedy|Family|Sci-Fi,Steve_Carell,Despicable_Me_2,high,high,Steve_Coogan,low,dating|minion|overprotective father|undercover...,high,English,USA,PG,high,high,high,high,low,high
1060,Color,Ted_Kotcheff,medium,low,high,medium,Brian_Dennehy,high,NaN,Action|Adventure|Drama,Sylvester_Stallone,First_Blood,high,high,Chris_Mulkey,low,1980s|colonel|deputy|rambo|sheriff,high,English,USA,R,medium,low,high,high,medium,low
4891,Color,Jon_Shear,low,medium,low,low,Josh_Hamilton,low,low,Drama,Dan_Futterman,Urbania,low,low,Matt_Keeslar,low,hot dog bun|needle|new york|rat|tourist,low,English,USA,R,low,low,low,medium,low,low
3710,Color,Patty_Jenkins,high,medium,high,high,Bruce_Dern,high,medium,Biography|Crime|Drama|Thriller,Charlize_Theron,Monster,high,high,Pruitt_Taylor_Vince,high,beach|female criminal|hooker|lesbian|prostitute,high,English,USA,R,low,medium,high,high,low,low
1765,Color,John_Glen,medium,high,medium,medium,Topol,medium,high,Action|Adventure|Thriller,Julian_Glover,For_Your_Eyes_Only,high,medium,Desmond_Llewelyn,low,agent|british|ice skater|ship|spy,high,English,UK,PG,medium,low,medium,medium,medium,high
4436,Color,Jaime_Zevallos,NaN,low,high,low,Jaime_Zevallos,low,NaN,Comedy|Drama|Romance,Angela_Sarafyan,Me_You_and_Five_Bucks,low,low,Michael_Elian,low,NaN,NaN,English,USA,NaN,low,high,low,high,NaN,low
1026,Color,Mark_Neveldine,high,low,medium,high,Logan_Lerman,high,medium,Action|Sci-Fi|Thriller,Gerard_Butler,Gamer,high,high,Alison_Lohman,low,battle|convict|death|mind control|role playing,medium,English,USA,R,high,medium,high,low,low,low
63,Color,David_Yates,high,medium,high,low,Alexander_Skarsgård,high,high,Action|Adventure|Drama|Romance,Christoph_Waltz,The_Legend_of_Tarzan,medium,high,Casper_Crump,high,africa|capture|jungle|male objectification|tarzan,medium,English,USA,PG-13,high,high,high,medium,medium,high


In [13]:
input = []
for a in df.iterrows():
    words1 = []
    label1 = []
    
    for b in a[1].iteritems():
        if(b[0] == 'movie_title'):
            label1.append(str(b[1]))
            words1.append(str(b[0]))
            words1.append(str(b[1]))
        elif b[0] == 'genres':
            for g in str(b[1]).split("|"):
                words1.append('genre')
                words1.append(trim_and_remove_spaces(g.lower()))
        elif b[0] == 'plot_keywords':
            for p in str(b[1]).split("|"):
                words1.append('plot')
                words1.append(trim_and_remove_spaces(p.lower()))
        elif b[0] in ['director_name', 'actor_2_name', 'actor_1_name', 'actor_3_name']:
            words1.append(str(b[0]))
            words1.append(str(b[1]))
        else:
            words1.append(str(b[0]) + "_" + str(b[1]))
    input.append(doc2vec.LabeledSentence(words=words1, tags=label1))

In [14]:
input

[LabeledSentence(words=['color_Color', 'director_name', 'Richard_Linklater', 'num_critic_for_reviews_high', 'duration_medium', 'director_facebook_likes_low', 'actor_3_facebook_likes_low', 'actor_2_name', 'Brandon_Smith', 'actor_1_facebook_likes_high', 'gross_low', 'genre', 'comedy', 'genre', 'crime', 'genre', 'drama', 'actor_1_name', 'Matthew_McConaughey', 'movie_title', 'Bernie', 'num_voted_users_medium', 'cast_total_facebook_likes_high', 'actor_3_name', 'Rick_Dial', 'facenumber_in_poster_low', 'plot', 'funeral', 'plot', 'stockbroker', 'plot', 'texas', 'plot', 'vacation', 'plot', 'widow', 'num_user_for_reviews_medium', 'language_English', 'country_USA', 'content_rating_PG-13', 'budget_low', 'title_year_high', 'actor_2_facebook_likes_low', 'imdb_score_medium', 'aspect_ratio_low', 'movie_facebook_likes_high'], tags=['Bernie']),
 LabeledSentence(words=['color_Color', 'director_name', 'Steve_Oedekerk', 'num_critic_for_reviews_low', 'duration_low', 'director_facebook_likes_high', 'actor_3_

In [ ]:
model = doc2vec.Doc2Vec(documents=input,
    dm=1, # distributed memory
    alpha=0.025,
    min_alpha=0.025,
    min_count = 0,
    workers = multiprocessing.cpu_count(),
    iter=10,
    size=128,
)  # use fixed learning rate
for epoch in range(10):
    model.train(input)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay

In [ ]:
model['For_Your_Eyes_Only']